# Coronafallzahlen Einfluss auf Klima in Europa

Fragestellung:
Hatten die Corona Fallzahlen einen kurzfristigen Einfluss auf die Klimaentwicklung in den Jahren 2020-2021? Wenn ja, führten diese Veränderungen zu einer kurzzeitigen stagnierung des Klimawandels, oder konnten diesem evtl. sogar entgegenwirken?

Datenstory:


# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import subprocess
import sys
import os
from zipfile import ZipFile
import cdsapi
import xarray as xr
import requests
import cfgrib


RuntimeError: ecCodes library not found using ['eccodes', 'libeccodes.so', 'libeccodes']

# Access Corona API
- aktuelle Fallzahlen
- vaccinations haben evtl auch einfluss?



In [17]:
response = requests.get('https://api.covid19api.com/total/country/switzerland')
counter = 0
for i in response.json():
    print("counter", counter)
    print(i)
    counter += 1

ConnectionError: HTTPSConnectionPool(host='api.covid19api.com', port=443): Max retries exceeded with url: /total/country/switzerland (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x166011a50>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

# Access Weather API

Mögliche Daten:
- Wind min / avg / max
- Temperature min / avg / max
- Pressure
- Sea Levels
- CO2
- Glacier Melting
- Niederschlag


API Installation guide on Mac OS: https://confluence.ecmwf.int/display/CKB/How+to+install+and+use+CDS+API+on+macOS

1.) Create Account: https://cds.climate.copernicus.eu/user/login?destination=%2F%23!%2Fhome

2.) confirm Email (Account used: user 19sean18@gmail.com, password fhnw2022)

3.) Get url and key for API from https://cds.climate.copernicus.eu/api-how-to under section "Install the CDS API key"
shopukld look like this: 
url: https://cds.climate.copernicus.eu/api/v2
key: 129216:95769c6b-af77-40e8-9a15-84f5f1b9dccb
--> key and url depends on userlogin

4.) create file ".cdsapirc" in home directory and copy url and key into file

5.) go to Website of Dataset and accept the license agreement for the dataset (needs to be done for each dataset)


In [ ]:
# Connect Client
c = cdsapi.Client()

# Set path for dowload
filename = "download.zip"
path = f"{os.getcwd()}/data/"

# retrieve the data from API
c.retrieve(
    'derived-near-surface-meteorological-variables',
    {
        'version': '2.0',
        'format': 'zip',
        'variable': 'rainfall_flux',
        'reference_dataset': 'cru',
        'year': '2018',
        'month': '01',
    },
    f"{path}{filename}")

# extract downloaded files
extracted_files = []
with ZipFile(f"{path}{filename}", "r") as file:
    for e in file.namelist():
        extracted_files.append(e)
    file.extractall(path)

# remove zip file
os.remove(f"{path}{filename}")

# print names of downloaded files
print(f"Downloaded the following files: {extracted_files}")

2022-05-04 17:24:52,723 INFO Welcome to the CDS
2022-05-04 17:24:52,724 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/derived-near-surface-meteorological-variables
2022-05-04 17:24:52,834 INFO Request is completed
2022-05-04 17:24:52,834 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data6/dataset-derived-near-surface-meteorological-variables-0c7c0a5d-1157-4bd6-83a9-54cde4301f03.zip to /Users/seancorrigan/Documents/GitHub/Klimadaten_Challenge_fhnw/data/download.zip (28.1M)
2022-05-04 17:24:53,592 INFO Download rate 37.1M/s  


Downloaded the following files: ['Rainf_WFDE5_CRU_201801_v2.0.nc']


In [18]:
# import netCDF4
# --> netCDF4 Library doesnt work on a macbook

In [ ]:
f"{path}{extracted_files[0]}"

'/Users/seancorrigan/Documents/GitHub/Klimadaten_Challenge_fhnw/data/Rainf_WFDE5_CRU_201801_v2.0.nc'

In [19]:
## Open nc file
ds = xr.open_dataset(f"{path}{extracted_files[0]}", engine = "scipy")

df = ds.to_dataframe()

TypeError: Error: /Users/seancorrigan/Documents/GitHub/Klimadaten_Challenge_fhnw/data/Rainf_WFDE5_CRU_201801_v2.0.nc is not a valid NetCDF 3 file
            If this is a NetCDF4 file, you may need to install the
            netcdf4 library, e.g.,

            $ pip install netcdf4
            

In [ ]:
df

Rainf
lat    lon     time                      
-89.75 -179.75 2018-01-01 00:00:00    0.0
               2018-01-01 01:00:00    0.0
               2018-01-01 02:00:00    0.0
               2018-01-01 03:00:00    0.0
               2018-01-01 04:00:00    0.0
...                                   ...
 89.75  179.75 2018-01-31 19:00:00    NaN
               2018-01-31 20:00:00    NaN
               2018-01-31 21:00:00    NaN
               2018-01-31 22:00:00    NaN
               2018-01-31 23:00:00    NaN

[192844800 rows x 1 columns]

# Einlesen in Influx DB

Format:
- Timestamp
- Tag: Measurement
- Tag: Country
- Value

Initialisierung: 
- Laden der Daten in Bluk Files (evtl. ein Backup, welches lokal abgelegt wurde als csv oder Datenbank Backup)

Aktualisierung:
- Check mit Datenbank, wann letzte aktualisiserung durchgeführt wurde.
- Check Verbindung mit API.
- Abruf der Daten seit letzter aktualisierung.
- Überprüfung ob Daten i.O.
- Formatieren der Daten in gewünschtes Format.
- Einlesen der Daten (Tagesweise)
